In [1]:
#Import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib as jb
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import mean_squared_error, r2_score
data = pd.read_csv("./Data/salary.csv")
data.head(20)



,Education,Experience,Location,Job_Title,Age,Gender,Salary
0,High School,8,Urban,Manager,63,Male,84620.053665
1,PhD,11,Suburban,Director,59,Male,142591.255894
2,Bachelor,28,Suburban,Manager,61,Female,97800.255404
3,High School,29,Rural,Director,45,Male,96834.671282
4,PhD,25,Urban,Analyst,26,Female,132157.786175
5,PhD,19,Rural,Director,27,Female,156312.936208
6,PhD,4,Rural,Director,60,Female,130567.649455
7,PhD,13,Suburban,Director,49,Female,148707.740759
8,Bachelor,20,Urban,Engineer,25,Female,95945.275429
9,PhD,14,Urban,Analyst,58,Female,133339.389682


# SPLITTING


In [2]:
features =['Education', 'Experience', 'Location', 'Job_Title',]

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X = data[features]
y=data["Salary"]
X.head(20)


,Education,Experience,Location,Job_Title
0,High School,8,Urban,Manager
1,PhD,11,Suburban,Director
2,Bachelor,28,Suburban,Manager
3,High School,29,Rural,Director
4,PhD,25,Urban,Analyst
5,PhD,19,Rural,Director
6,PhD,4,Rural,Director
7,PhD,13,Suburban,Director
8,Bachelor,20,Urban,Engineer
9,PhD,14,Urban,Analyst


# CHUAN HOA

In [4]:
from sklearn.preprocessing import OrdinalEncoder 
EncoderEdu = OrdinalEncoder(categories=[['High School', 'PhD','Bachelor', 'Master']]) 
EncoderLoc = OrdinalEncoder(categories=[['Urban','Suburban', 'Rural']])
EncoderJob = OrdinalEncoder(categories=[['Analyst','Director','Manager','Engineer']])                                 
X['Education']= EncoderEdu.fit_transform(X[['Education']])
X['Location'] = EncoderLoc.fit_transform(X[['Location']])
X['Job_Title'] = EncoderJob.fit_transform(X[['Job_Title']])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X.head(20)


C:\Users\tienc\AppData\Local\Temp\ipykernel_13364\1208883054.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Education']= EncoderEdu.fit_transform(X[['Education']])
C:\Users\tienc\AppData\Local\Temp\ipykernel_13364\1208883054.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Location'] = EncoderLoc.fit_transform(X[['Location']])
C:\Users\tienc\AppData\Local\Temp\ipykernel_13364\1208883054.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

,Education,Experience,Location,Job_Title
0,0.0,8,0.0,2.0
1,1.0,11,1.0,1.0
2,2.0,28,1.0,2.0
3,0.0,29,2.0,1.0
4,1.0,25,0.0,0.0
5,1.0,19,2.0,1.0
6,1.0,4,2.0,1.0
7,1.0,13,1.0,1.0
8,2.0,20,0.0,3.0
9,1.0,14,0.0,0.0


# LINEAR REGRESSION

In [5]:
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
jb.dump(lr_model, 'linear.pkl')

['linear.pkl']

# Tìm alpha tối ưu


In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
# Thiết lập các giá trị alpha cần thử nghiệm
alphas = {'alpha': [0.001, 0.01, 0.1, 1, 10]}

# Tạo đối tượng Lasso và GridSearchCV
lasso = Lasso()
grid = GridSearchCV(lasso, alphas, cv=5)

# Tìm alpha tối ưu
grid.fit(X_train, y_train)
print(grid.best_params_)  # In ra alpha tốt nhất


{'alpha': 10}


# Lasso


In [7]:
from sklearn.linear_model import Lasso
lasso_model = Lasso(alpha=10)
lasso_model.fit(X_train, y_train)
y_pred_lasso = lasso_model.predict(X_test)
jb.dump(lasso_model, 'lasso.pkl')

['lasso.pkl']

# Neural Network

In [8]:
nn_model = MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=500)
nn_model.fit(X_train, y_train)
y_pred_nn = nn_model.predict(X_test)
jb.dump(nn_model, 'nn.pkl')

c:\Users\tienc\Documents\ML2\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


['nn.pkl']

# Stacking


In [9]:
from sklearn.ensemble import StackingRegressor
# Khởi tạo các mô hình thành phần
base_models = [
    ('lr', LinearRegression()),
    ('lasso', Lasso(alpha=10)),
    ('nn', MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=500))
]

# Mô hình tổng hợp
stacking_model = StackingRegressor(estimators=base_models, final_estimator=LinearRegression())

# Huấn luyện mô hình Stacking
stacking_model.fit(X_train, y_train)
y_pred_stacking = stacking_model.predict(X_test)
jb.dump(stacking_model, 'stacking.pkl')

c:\Users\tienc\Documents\ML2\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\tienc\Documents\ML2\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\tienc\Documents\ML2\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\tienc\Documents\ML2\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\tienc\Documents\ML2\.ve

['stacking.pkl']

# Danh gia

In [10]:
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error, mean_absolute_error

def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    return mse, r2, rmse, mae

# Đánh giá các mô hình
for model_name, y_pred in [("Linear Regression", y_pred_lr),
                           ("Lasso", y_pred_lasso),
                           ("Neural Network", y_pred_nn),
                           ("Stacking", y_pred_stacking)]:
    mse, r2, rmse, mae = evaluate_model(y_test, y_pred)
    print(f"{model_name} - MSE: {mse:.4f}, R²: {r2:.4f}, RMSE:{rmse:.4f}, MAE:{mae:.4f}")


Linear Regression - MSE: 614351679.2189, R²: 0.2476, RMSE:24786.1187, MAE:19969.1471
Lasso - MSE: 614299293.1520, R²: 0.2477, RMSE:24785.0619, MAE:19970.9014
Neural Network - MSE: 603679395.3940, R²: 0.2607, RMSE:24569.8880, MAE:19886.4454
Stacking - MSE: 613679369.2956, R²: 0.2484, RMSE:24772.5527, MAE:19859.0479
